In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Load the data
df = pd.read_csv('data.csv')
df.head()

# Convert the predictor variables and the target of the data to arrays. 
X = df.iloc[:, 2:32].to_numpy()
X

y1 = df.iloc[:, 1]
y = LabelEncoder().fit_transform(y1)
y

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = (y != 0).astype(int)  # Convert to binary classification problem

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(30, 60) ## the actual number of the predictor variables
        self.fc2 = nn.Linear(60, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Instantiate the neural network
net = Net()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

# Train the neural network
for epoch in range(100):
    optimizer.zero_grad()
    outputs = net(torch.Tensor(X_train).float())
    loss = criterion(outputs, torch.Tensor(y_train).float().view(-1, 1))
    loss.backward()
    optimizer.step()

# Test the neural network
with torch.no_grad():
    outputs = net(torch.Tensor(X_test).float())
    predicted = (outputs > 0.5).float()
    accuracy = (predicted == torch.Tensor(y_test).float().view(-1, 1)).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.9824561476707458


In [53]:
# RMSE calculation
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, predicted, squared=False)
print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 0.132
